<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# NeurIPS - Run Images Min sigma

Goal : have well trained models for image sampling

Data : celeba64,mnist,svhn,celeba32

Models : ConvLNPXL 

Loss : NLLLloss

Runs : 1

In [1]:
import os

os.chdir("..")

In [2]:
import submitit

from train_imgs import main, parse_arguments
from results import get_exp_args

In [3]:

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train = True
        return submitit.utils.DelayedSubmission(self, args)

    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)


In [4]:
args = get_exp_args("exp_images_minsig", is_load=False)
len(args)

7

In [5]:
args_64 = [arg for arg in args if arg.dataset=="celeba64"]
args_32 = [arg for arg in args if arg.dataset!="celeba64"]

In [7]:
executor_64=submitit.SlurmExecutor(folder="logs/%j", max_num_timeout=3)
executor_64.update_parameters(num_gpus=1, 
                           time=60*24*2,  
                           cpus_per_task=10, 
                           mem='32GB',
                          partition="priority",
                           comment="neurips",
                           constraint="volta32gb"
                          )

In [8]:
executor_32=submitit.SlurmExecutor(folder="logs/%j", max_num_timeout=3)
executor_32.update_parameters(num_gpus=1, 
                           time=60*24*2,  
                           cpus_per_task=10, 
                           mem='32GB',
                          partition="priority",
                           comment="neurips"
                          )

In [9]:
jobs_32 = executor_32.map_array(Run(), args_32)

In [10]:
jobs_64 = executor_64.map_array(Run(), args_64)

In [11]:
############################################################

In [22]:
jobs_32

[SlurmJob<job_id=26816891_0, task_id=0, state="RUNNING">,
 SlurmJob<job_id=26816891_1, task_id=0, state="RUNNING">,
 SlurmJob<job_id=26816891_2, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26816891_3, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26816891_4, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26816891_5, task_id=0, state="COMPLETED">]

In [21]:
jobs_64

[SlurmJob<job_id=26816892_0, task_id=0, state="RUNNING">]

In [19]:
for job in jobs_32:
    print("--------------------------------")
    print(job.stdout())

--------------------------------
submitit INFO (2020-05-25 23:31:11,034) - Starting with JobEnvironment(job_id=26816891_0, hostname=learnfair0222, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-05-25 23:31:11,034) - Loading pickle: /private/home/yannd/projects/NPF/logs/26816891_0/26816891_0_submitted.pkl

--- Training celeba32/ConvNPFXL_NllLNPF/run_0 ---

  epoch    train_loss    valid_loss    cp        dur
-------  ------------  ------------  ----  ---------
      1    -2396.2040    -2970.1083     +  1684.7565
      2    -3073.2474    -3526.9835     +  1607.1930
      3    -3405.1207    -3546.4629     +  1590.9843
      4    -3596.5188    -3718.3481     +  1590.3416
      5    -3722.8393    -3830.5934     +  1589.1623
      6    -3805.7429    -4001.2037     +  1590.5324
      7    -3875.2835    -3902.6061        1599.1967
      8    -3934.8339    -4039.5530     +  1612.9628
      9    -3950.6894    -4029.4130        1601.4513
     10    -3994.0340    -4030.6578     

In [18]:
for j in jobs_32:
    print("--------------------------------")
    print(j.stderr())

--------------------------------

--------------------------------
/private/home/yannd/projects/NPF/neuralproc/architectures/mlp.py:67: UserWarning: hidden_size=32 smaller than output=128 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))

--------------------------------
INFO:submitit:Job completed successfully

--------------------------------
/private/home/yannd/projects/NPF/neuralproc/architectures/mlp.py:67: UserWarning: hidden_size=32 smaller than output=128 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
INFO:submitit:Job completed successfully

--------------------------------
INFO:submitit:Job completed successfully

--------------------------------
/private/home/yannd/projects/NPF/neuralproc/architectures/mlp.py:67: UserWarning: hidden_size=32 smaller than output=128 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, o

In [21]:
for job in jobs_64:
    job.cancel()